In [11]:
import os

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# from tftrainer import Trainer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    confusion_matrix,
    classification_report
)

## Cargar los datos y dividirlos en training y validation

In [12]:
df = pd.read_csv('./data/goemotions_clean.csv', sep=",")
df.head()

,text,emotion
0,Shhh dont give idea,anger
1,Thank much kind stranger I really need,gratitude
2,Ion know would better buy trim make hard dose,neutral
3,Im honestly surprised We fallen much farther,excitement
4,Jurisprudence fetishist get technicality,neutral


In [13]:
# Dividir el dataset en train y validation
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['emotion'], test_size=0.2, random_state=0)

## Preprocesamiento de los datos

In [15]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [16]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder

class PyTorchDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels.values.reshape(-1, 1)
        
        # One-hot encode the labels
        self.encoder = OneHotEncoder()
        self.labels_encoded = self.encoder.fit_transform(self.labels).toarray() 
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        sample = {
            'input_ids': torch.tensor(self.inputs[idx]),
            'labels': torch.tensor(self.labels_encoded[idx], dtype=torch.float32)  # Use float32 for binary labels
        }
        return sample



In [17]:
# Tokenize train and test sets
X_train_tokenized = tokenizer(X_train.tolist(), truncation=True, padding=True)
X_val_tokenized = tokenizer(X_val.tolist(), truncation=True, padding=True)

# Create PyTorch datasets
train_dataset = PyTorchDataset(X_train_tokenized["input_ids"], y_train)
test_dataset = PyTorchDataset(X_val_tokenized["input_ids"], y_val)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=23, 
    problem_type="multi_label_classification")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_steps = 10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [20]:
trainer.train()

  0%|          | 10/4038 [01:31<9:47:17,  8.75s/it]

{'loss': 0.6918, 'grad_norm': 0.6968169212341309, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  0%|          | 20/4038 [03:01<9:49:42,  8.81s/it] 

{'loss': 0.6815, 'grad_norm': 0.7035000324249268, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


  1%|          | 30/4038 [04:29<9:38:34,  8.66s/it] 

{'loss': 0.6621, 'grad_norm': 0.7568291425704956, 'learning_rate': 3e-06, 'epoch': 0.02}


  1%|          | 40/4038 [06:03<10:47:00,  9.71s/it]

{'loss': 0.6297, 'grad_norm': 0.8305456042289734, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}


  1%|          | 46/4038 [07:06<11:57:09, 10.78s/it]